In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import scipy.spatial as sp
import unicodedata
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import word2vec
import pprint
import numpy as np
from scipy import spatial

mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

vector_size=250

In [3]:
'''
csvの読み込み
電波途切れた関係で100店舗のもの（../csv/review_100.csv）とそれ以外の全てのデータ(../csv/review.csv)が存在する
これをあとでマージ
'''

# review_100 = pd.read_csv('../csv/review_100.csv') # shape(99,8)
# review_other = pd.read_csv('../csv/review.csv') # shape(1000,8)

'\ncsvの読み込み\n電波途切れた関係で100店舗のもの（../csv/review_100.csv）とそれ以外の全てのデータ(../csv/review.csv)が存在する\nこれをあとでマージ\n'

In [4]:
# review_df = pd.concat([review_100, review_other]) # shape(1099, 8)
# review_df

In [5]:
# review_df = review_df.drop(['Unnamed: 0', 'pref', 'review_cnt'], axis=1)  

In [6]:
# review_df = review_df.set_index('store_id')

In [7]:
# review_df.to_csv('../csv/review_new.csv')

In [8]:
review_df = pd.read_csv('/Users/kazuki_takahashi/Desktop/python_study/datamix_ramen/csv/review.csv')

In [9]:
# review_df.score.value_counts().sort_index().plot(kind='bar')

In [31]:
review_df['station'] = review_df['station'].str.rstrip('駅') # 駅という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('円','') # 円という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('点','') # 点という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('時','') # 時という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('分','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('!','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('・','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace(';','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('?','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('年','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('月','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('日','') # 分という文字列を削除して格納し直す
review_df['review'] = review_df['review'].str.replace('゚ Д ゚','') # 分という文字列を削除して格納し直す
review_df

,store_id,store_name,score,station,review,wakati_review
0,1,中華そば しば田,4.06,仙川,開店前から長蛇の列( ロ)゚ ゚開店前10前に到着しましたが、ありつけるまで、1間半を要し...,開店 前 長蛇 列 ロ ゚ ゚ 開店 前 前 到着 間 半 そこ 東京 規模 の 夜 煮干し...
1,2,Homemade Ramen 麦苗,4.02,大森海岸,わがままボディーの後輩との東京遠征2目の昼。実に4ぶりとなるこちらへ。当から行列のできるラー...,わがまま ボディー 後輩 東京 遠征 目 昼 ぶり こちら 当 行列 ラーメン屋 食べログ ...
2,3,手打式超多加水麺 ののくら,3.98,亀有,2020128 初訪TRY本で新人賞を獲った頃から行きたかった「ののくら」。今の午前中の打ち...,訪 TRY 本 新人賞 頃 のの くら 今 午前中 打ち合わせ 新小岩 午後 予定 く間 余...
3,4,中華そば 多賀野,3.97,荏原中延,中華そば750、大盛120動物系乾物系背脂醤油スープ中細ストレート麺ちょいカタメ豚バラチャー...,中華そば 大盛 動物 系 乾物 系 背脂 醤油 スープ 中 ストレート麺 カタメ 豚バラ チ...
4,5,麺庵ちとせ,3.97,若松河田,百名店ラーメンTOKYO2019の51店目は曙橋の『麺庵ちとせ』さんへ地下鉄の駅からお店へと...,百名 店 ラーメン TOKYO 店 目 曙 橋 麺 庵 ちとせ さん 地下鉄 駅 店 坂道 ...
...,...,...,...,...,...,...
1094,1096,旨辛ラーメン 表裏,3.50,市ケ谷,市ヶ谷ランチシリーズ(笑)まぁいつも通りと言うか麺屋さんから攻めていくわけですね〜こちらのお...,市ヶ谷 ランチ シリーズ 笑 通り 麺 屋 さん わけ こちら 店 ん カップラーメン 結構...
1095,1097,麺屋 ほたる,3.50,板橋,板橋駅の近くにお店があります。このは昼に煮干しラーメンを食べたので繋げて魚介系のつけ麺を食べ...,板橋駅 近く 店 このは 昼 ラーメン 魚介 系 つけ麺 こと 板橋駅 徒歩 到着 裏通り ...
1096,1098,中華そば ばんや,3.50,下高井戸,東京都世田谷区松原。無化調、麺スープチャーシュー水等全てを厳選、こだわり抜いた中華そば店です...,東京都世田谷区松原 無化調 麺 スープ チャーシュー 水 等 全て 厳選 中華そば 店 最寄...
1097,1099,こうや麺房,3.50,本郷三丁目,クライアントの方から少し早めのランチの誘い狙いは前回訪問で好評だった江戸富士お店に向うと12...,クライアント 方 早め ランチ 誘い 狙い 前回 訪問 好評 江戸 富士 店 前 以上 並び...


In [11]:
review_df['review'] = review_df['review'].str.normalize('NFKC')

In [12]:
def get_noun(text):
    mecab.parse('')
    node = mecab.parseToNode(text)
    result = []

    while node:
        word = node.surface
        pos,pos2 = node.feature.split(",")[0],node.feature.split(",")[1]
        if '0' in word or '1' in word or '2' in word or '3' in word or '4' in word or '5' in word or '6' in word or '7' in word or '8' in word or '9' in word:
            node = node.next
            continue

        if pos2 == '数':
            node = node.next
            continue
            
        if pos == '名詞' and pos2 != '数':
            result.append(word)

        node = node.next

    return ' '.join(result)

In [13]:
review_df['wakati_review'] = review_df['review'].apply(get_noun)

In [14]:
# review_df['wakati_review'][:10]

In [15]:
vectrizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [16]:
X = vectrizer.fit_transform(review_df['wakati_review'])

In [17]:
X[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [18]:
# review_df.head()

In [19]:
sentences = [token.split() for token in review_df.wakati_review]

In [20]:
cbow_model = word2vec.Word2Vec(
    sentences,
    sg=0,
    size=250,
    min_count=10,
    window=15,
    seed=1234
)

In [21]:
cbow_model.save('/Users/kazuki_takahashi/Desktop/python_study/datamix_ramen/models/v2_cbow_w2v.model')

In [22]:
feature_names = vectrizer.get_feature_names()
# vectrizer.get_feature_names()
# X
feature_names

['a',
 'aa',
 'aazokk',
 'ab',
 'aba',
 'abc',
 'abe',
 'abelog',
 'abematv',
 'about',
 'aburasso',
 'aburraso',
 'ac',
 'academicseriesnewasia',
 'acc',
 'accept',
 'ace',
 'acm',
 'aco',
 'acuo',
 'ad',
 'adap',
 'additional',
 'adores',
 'adp',
 'adsbygoogle',
 'ae',
 'aepvkae',
 'af',
 'afk',
 'afn',
 'afri',
 'after',
 'afu',
 'afuri',
 'again',
 'against',
 'ah',
 'ahead',
 'ahref',
 'aid',
 'aiming',
 'aji',
 'ajibana',
 'ajito',
 'ajitoism',
 'ajzvnpcx',
 'ak',
 'aka',
 'akane',
 'akanoren',
 'akasak',
 'akasaka',
 'akasakashare',
 'akb',
 'akind',
 'akira',
 'aky',
 'alacarte',
 'alc',
 'ale',
 'alexa',
 'algy',
 'alice',
 'all',
 'allstar',
 'aloha',
 'alps',
 'alqod',
 'also',
 'alwaysudon',
 'am',
 'amazing',
 'amazon',
 'amazonプライム',
 'ameba',
 'ameblo',
 'amemberentry',
 'amemiya',
 'american',
 'amp',
 'ampampampampampampquot',
 'ampampampquot',
 'an',
 'ana',
 'and',
 'angel',
 'ani',
 'anniversary',
 'answer',
 'anyway',
 'anzc',
 'aoixcqdgpu',
 'aoki',
 'aor',
 'aoya

In [34]:
def avg_feature_vectors(words, model):
    feature_vec = np.zeros(vector_size, dtype='float32')
    for word in words:
        try:
            feature_vec = np.add(feature_vec, model[word])
        except KeyError:
            # print(word)
            pass

    return feature_vec

In [35]:
avg_feature_vectors(["みそ", '醬油'], cbow_model)

/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


array([ 1.5006418e+00,  1.4267657e+00, -1.2165003e+00,  1.8332287e+00,
       -2.4734023e-01, -1.7145119e+00, -5.4836458e-01,  1.4006644e+00,
        2.5735673e-01, -3.7496507e-02, -3.0859530e-01, -1.8600399e+00,
        5.0292140e-01, -1.2265253e+00,  1.1540104e+00, -2.5370295e+00,
        2.7201480e-01,  1.5163113e+00, -1.2764060e+00, -1.4737046e+00,
        6.3026011e-02, -2.3077202e+00,  1.5723908e-01,  5.1920080e-01,
       -2.6685888e-01, -5.3555250e-01, -2.4342296e+00, -1.0829518e+00,
       -5.5123711e-01,  8.9728838e-01,  2.9415092e-01, -2.0303862e+00,
        3.4530807e+00, -6.3203037e-01,  1.5890864e-01, -1.4407407e+00,
       -2.1525063e-01,  1.7000341e-01,  1.6810092e-01,  1.4574926e+00,
        5.3737819e-01,  5.4259586e-01,  9.0763688e-01, -5.1547664e-01,
       -1.3292191e+00, -1.6233336e+00,  1.1925880e+00, -1.0420419e+00,
        7.2341394e-01,  1.0342604e+00, -1.1196646e-01,  3.2398146e-01,
        1.6428232e-01,  3.4621358e-03, -5.9493065e-01,  1.6565794e-01,
      

In [27]:
for review in review_df['wakati_review']:
    review_sentence = review.split(" ")
    vec = avg_feature_vectors(review_sentence, cbow_model)
    
vec



一枚目
柴田氏
柴田氏
成田
福一
琴線
柴田氏
柴田
手つかず
特攻
直射
武州
轟
格式
尻すぼみ
手抜かり
くせになりそう
足代
王様のブランチ
干しシイタケ
柴田
花嵐
コラボメニュー
軟化
大口
高野
データー
三位
ウマッ
ウマッ
特番
末尾
書籍
OPPORTUNITY
エベンペーガン
kaolianmarket
ik
トピック
淀み
鴨地
よすぎ
おとこ
玉水
サーバ
バーブ
新百合ヶ丘
パツッ
コリッ
夫々
幹
価格差
支払額
粉唐辛子
タイムラグ
手許
何食わぬ顔
仰天
洋式
男女兼用
ゆう
get
スゲェ
指先
サクラ色
とだな
/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


array([-4.56192871e+02, -5.10671051e+02, -8.40952087e+02,  4.24227448e+02,
        5.51662415e+02, -5.23827820e+02, -6.19338745e+02,  2.75907745e+02,
        5.56877563e+02, -7.42640198e+02,  3.96572479e+02, -1.42772369e+02,
        5.88770409e+01,  9.69283936e+02, -2.09881180e+02, -1.78751968e+02,
        3.42962524e+02, -7.33821045e+02,  1.23623268e+02,  3.80822052e+02,
        3.89262817e+02, -1.04564075e+03, -4.97311401e+02,  2.24252453e+01,
       -5.05028595e+02, -2.36286926e+02,  7.36947861e+01, -9.85929504e+02,
       -8.62322021e+02,  2.54878723e+02, -2.94256927e+02, -6.12469055e+02,
        1.52957983e+03, -1.63201523e+02,  4.24608032e+02, -6.46821167e+02,
       -1.30639722e+03,  5.73638992e+01,  1.28503891e+02,  8.04398865e+02,
        2.90475067e+02, -3.95582123e+02,  7.57942749e+02,  3.61537415e+02,
       -4.58546967e+02,  2.07787079e+02, -4.71953125e+02, -2.27718544e+01,
        3.20369812e+02,  1.76172272e+02,  1.87793167e+02,  4.78113434e+02,
        8.05512329e+02,  

In [38]:
def sentence_similarity(sentence1, sentence2):
    sentence1_avg_vector = avg_feature_vectors(sentence1, cbow_model)
    sentence2_avg_vector = avg_feature_vectors(sentence2, cbow_model)

    return 1 - spatial.distance.cosine(sentence1_avg_vector, sentence2_avg_vector)

In [49]:
# result = sentence_similarity(review_df['wakati_review'][5], ['煮干し', '中華そば'])

/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [58]:
result = []

In [76]:
for review in review_df['wakati_review']:
    score = sentence_similarity(review, ['醬油'])
    result.append(score*100)

/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [77]:
l = sorted(result, reverse=True)
l[0]

42.68302321434021

In [78]:
for review in review_df['wakati_review']:
    score = sentence_similarity(review, ['味噌'])
    result.append(score*100)

/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
